In [9]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
import MathOptInterface as MOI
import Ipopt 
import FiniteDiff
import ForwardDiff as FD
import Convex as cvx 
import ECOS
import MeshCat as mc
import Distributions
import Random

using LinearAlgebra
using Plots
using Random
using JLD2
using Test


  Activating project at `c:\Users\Daniel\Documents\Python-Projects\AiPEX-Projects\warmstarting_NLPs`


In [10]:
include(joinpath(@__DIR__, "utils","fmincon.jl"))
include(joinpath(@__DIR__, "utils","cartpole_animation.jl"))

animate_cartpole (generic function with 1 method)

In [11]:
# cartpole 
function dynamics(params::NamedTuple, x::Vector, u)
    # cartpole ODE, parametrized by params. 

    # cartpole physical parameters 
    mc, mp, l = params.mc, params.mp, params.l
    g = 9.81
    
    q = x[1:2]
    qd = x[3:4]

    s = sin(q[2])
    c = cos(q[2])

    H = [mc+mp mp*l*c; mp*l*c mp*l^2]
    C = [0 -mp*qd[2]*l*s; 0 0]
    G = [0, mp*g*l*s]
    B = [1, 0]

    qdd = -H\(C*qd + G - B*u[1])
    xdot = [qd;qdd]
    return xdot 

end

function hermite_simpson(params::NamedTuple, x1::Vector, x2::Vector, u, dt::Real)::Vector
    # TODO: input hermite simpson implicit integrator residual 
     x_mid = 0.5(x1 + x2) + (dt/8) * (dynamics(params, x1, u) - dynamics(params, x2, u))
     res = x1 + (dt/6) * (dynamics(params, x1, u) + 4*dynamics(params, x_mid, u) + dynamics(params, x2, u)) - x2
     return res
end

hermite_simpson (generic function with 1 method)

In [12]:
function create_idx(nx,nu,N)
    # This function creates some useful indexing tools for Z 
    # x_i = Z[idx.x[i]]
    # u_i = Z[idx.u[i]]
    
    # Feel free to use/not use anything here.
    
    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z 
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
    
    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx 
    
    return (nx=nx,nu=nu,N=N,nz=nz,nc=nc,x= x,u = u,c = c)
end

function cartpole_cost(params::NamedTuple, Z::Vector)::Real
    idx, N, xg = params.idx, params.N, params.xg
    Q, R, Qf = params.Q, params.R, params.Qf
    
    # TODO: input cartpole LQR cost 
    J = 0 

    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]
       
        J += 0.5*(xi-xg)'*Q*(xi-xg) + 0.5*ui'*R*ui
    end
    
    # dont forget terminal cost 
    xN = Z[idx.x[N]]
    J += 0.5*(xN-xg)'*Qf*(xN-xg)
    return J 
end

function cartpole_dynamics_constraints(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    
    # TODO: create dynamics constraints using hermite simpson 

    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), idx.nc)
    
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]] 
        xip1 = Z[idx.x[i+1]]
        
        # TODO: hermite simpson 
        c[idx.c[i]] = hermite_simpson(params, xi, xip1, ui, dt)
    end
    return c 
end

function cartpole_equality_constraint(params::NamedTuple, Z::Vector)::Vector
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 
    
    
    # TODO: return all of the equality constraints 

    
    return [Z[idx.x[1]] - xic; Z[idx.x[end]] - xg; cartpole_dynamics_constraints(params, Z)] 
end

function solve_cartpole_swingup(σ; verbose=true)
    
    # problem size 
    nx = 4 
    nu = 1 
    dt = 0.05
    tf = 2.0 
    t_vec = 0:dt:tf 
    N = length(t_vec)
    
    # LQR cost 
    Q = diagm(ones(nx))
    R = 0.1*diagm(ones(nu))
    Qf = 10*diagm(ones(nx))
    
    # indexing 
    idx = create_idx(nx,nu,N)
    
    # initial and goal states 
    # xic = [0, 0, 0, 0]
    xic = [σ[1], σ[2], 0, 0]
    xg = [0, pi, 0, 0]
    
    # load all useful things into params 
    params = (Q = Q, R = R, Qf = Qf, xic = xic, xg = xg, dt = dt, N = N, idx = idx,mc = 1.0, mp = 0.2, l = 0.5)
    
    # TODO: primal bounds 
    x_l = fill(-Inf, idx.nz)
    x_u = fill(Inf, idx.nz)
    
    for i = 1:(N-1)
        x_l[idx.u[i]] .= -10
        x_u[idx.u[i]] .= 10
    end

    
    # inequality constraint bounds (this is what we do when we have no inequality constraints)
    c_l = zeros(0)
    c_u = zeros(0)
    function inequality_constraint(params, Z)
        return zeros(eltype(Z), 0)
    end
    
    # initial guess 
    z0 = 0.001*randn(idx.nz)
    
    # choose diff type (try :auto, then use :finite if :auto doesn't work)
    diff_type = :auto 
#     diff_type = :finite
    
    # @show cartpole_equality_constraint(params, z0)
    # @show cartpole_dynamics_constraints(params, z0)
    # @show inequality_constraint(params, z0)
    # @show cartpole_cost(params, z0)
        
    Z, obj, solve_time_sec, term_status = fmincon(cartpole_cost,cartpole_equality_constraint,inequality_constraint,
                x_l,x_u,c_l,c_u,z0,params, diff_type;
                tol = 1e-6, c_tol = 1e-6, max_iters = 10_000, verbose = verbose)
    # term_status = 0
\



    # pull the X and U solutions out of Z 
    X = [Z[idx.x[i]] for i = 1:N]
    U = [Z[idx.u[i]] for i = 1:(N-1)]
    
    return X, U, obj, solve_time_sec, term_status, t_vec, params 
end


    


solve_cartpole_swingup (generic function with 1 method)

In [13]:
# X, U, obj, t_vec, params = solve_cartpole_swingup(verbose=true)

# # --------------testing------------------

# Xm = hcat(X...)
# Um = hcat(U...)

# # --------------plotting-----------------
# display(plot(t_vec, Xm', label = ["p" "θ" "ṗ" "θ̇"], xlabel = "time (s)", title = "State Trajectory"))
# display(plot(t_vec[1:end-1],Um',label="",xlabel = "time (s)", ylabel = "u",title = "Controls"))

# # display(animate_cartpole(X, 0.05))



## Solve the DIRCOL NLP for a parameter set

In [16]:
## Define upper and lower bounds of the parameters for the Paramaetric Optimal Control Problem
# using the xic of the cartpole
using Random, Distributions, CSV, DataFrames
Random.seed!(123)


N = 5000 # number of samples
σ_lower = [0.0, 0.0]
σ_upper = [0.2, pi/2]

# Randomly sample the iid parameters uniformly from the given bounds
d = Product(Uniform.(σ_lower, σ_upper))
σ_samples = rand(d, N)
σ_samples = eachcol(σ_samples)

# Solve the NLP for the parameter sample set
df = DataFrame(params = Vector{Vector{Float64}}(), X=Vector{Vector{Vector{Float64}}}(), U=Vector{Vector{Vector{Float64}}}(), obj = Float64[], solve_time_sec = Float64[], term_status = MOI.TerminationStatusCode[])

i = 1
for σ in σ_samples
    println("Sample: ", i)
    println("------------------")
    println("σ: ", σ)
    X, U, obj, solve_time_sec, term_status, t_vec, params = solve_cartpole_swingup(σ, verbose=false)
    println("Objective Value: ", obj)
    println("TerminationStatusCode: ", term_status)
    println("")

    push!(df, [σ, X, U, obj, solve_time_sec, term_status])
    i += 1
end



Sample: 1
------------------
σ: [0.10424275910707662, 0.9217538991461435]
Objective Value: 450.37534761715335
TerminationStatusCode: LOCALLY_SOLVED

Sample: 2
------------------
σ: [0.17817573961855623, 0.29987554159020713]
Objective Value: 403.572057111036
TerminationStatusCode: LOCALLY_SOLVED

Sample: 3
------------------
σ: [0.10513247830840947, 0.6135346283367087]
Objective Value: 421.8986475928782
TerminationStatusCode: LOCALLY_SOLVED

Sample: 4
------------------
σ: [0.008963601003498222, 1.4661079152569128]
Objective Value: 519.1632438761235
TerminationStatusCode: LOCALLY_SOLVED

Sample: 5
------------------
σ: [0.11611199637490824, 0.5140240587239221]
Objective Value: 414.5490586426166
TerminationStatusCode: LOCALLY_SOLVED

Sample: 6
------------------
σ: [0.1053991837593973, 1.3135447740528496]
Objective Value: 498.2110700042299
TerminationStatusCode: LOCALLY_SOLVED

Sample: 7
------------------
σ: [0.00818122720553851, 0.7307368158722805]
Objective Value: 431.98989756232606
T

In [20]:

CSV.write("data/cartpole_DIRCOL.csv", df)

df

Row,params,X,U,obj,solve_time_sec,term_status
,Array…,Array…,Array…,Float64,Float64,Terminat…
1,"[0.104243, 0.921754]","[[0.104243, 0.921754, 0.0, 0.0], [0.11345, 0.891086, 0.369608, -1.2282], [0.138551, 0.802418, 0.637295, -2.31869], [0.174667, 0.663432, 0.80984, -3.23289], [0.216977, 0.484831, 0.882279, -3.88846], [0.260568, 0.281416, 0.857201, -4.20903], [0.301161, 0.0702691, 0.760036, -4.18903], [0.336202, -0.133298, 0.635649, -3.90885], [0.365366, -0.318818, 0.527057, -3.47646], [0.390064, -0.480593, 0.459106, -2.96838] … [-0.174317, 2.53509, 0.016394, 2.15014], [-0.168316, 2.63805, 0.22442, 1.98761], [-0.153182, 2.73327, 0.381754, 1.83893], [-0.131381, 2.8212, 0.491207, 1.69456], [-0.105259, 2.90184, 0.554589, 1.54588], [-0.0771291, 2.97469, 0.571482, 1.3816], [-0.0494142, 3.03853, 0.537951, 1.18396], [-0.0248626, 3.09099, 0.444848, 0.924087], [-0.0068749, 3.12779, 0.275216, 0.554925], [0.0, 3.14159, 0.0, 2.3038e-19]]","[[7.32661], [4.74569], [2.26965], [-0.0478557], [-1.80822], [-2.60241], [-2.37622], [-1.46033], [-0.292504], [0.804584] … [6.25694], [5.04411], [3.88234], [2.7857], [1.73449], [0.683972], [-0.435327], [-1.72491], [-3.3351], [-5.49545]]",450.375,0.894,LOCALLY_SOLVED
2,"[0.178176, 0.299876]","[[0.178176, 0.299876, 0.0, 0.0], [0.184596, 0.280428, 0.256768, -0.775276], [0.202713, 0.225058, 0.467569, -1.43101], [0.230646, 0.139736, 0.648708, -1.96705], [0.267135, 0.0307534, 0.808877, -2.37113], [0.311189, -0.0947752, 0.950548, -2.62381], [0.361822, -0.228715, 1.07167, -2.70526], [0.417878, -0.362116, 1.16799, -2.60371], [0.477998, -0.485816, 1.23522, -2.32184], [0.540628, -0.591173, 1.26953, -1.87625] … [-0.111005, 2.79488, 0.202832, 1.4749], [-0.0990895, 2.86421, 0.274551, 1.31095], [-0.0843419, 2.92551, 0.31611, 1.15264], [-0.0681484, 2.97909, 0.332347, 1.00058], [-0.0516831, 3.02524, 0.326922, 0.854112], [-0.0359692, 3.06417, 0.302213, 0.710621], [-0.0219469, 3.09591, 0.259168, 0.564947], [-0.0105519, 3.12012, 0.197011, 0.408413], [-0.00281535, 3.13594, 0.112706, 0.227256], [0.0, 3.14159, 0.0, -4.86337e-26]]","[[4.67247], [3.75938], [3.24142], [2.99978], [2.91601], [2.86138], [2.71162], [2.37679], [1.81419], [1.00081] … [2.80673], [1.97385], [1.26743], [0.66654], [0.147922], [-0.313318], [-0.744817], [-1.17968], [-1.66119], [-2.25045]]",403.572,1.356,LOCALLY_SOLVED
3,"[0.105132, 0.613535]","[[0.105132, 0.613535, 0.0, 0.0], [0.112468, 0.58747, 0.293961, -1.04142], [0.132124, 0.513871, 0.493121, -1.89624], [0.159943, 0.40214, 0.619538, -2.55855], [0.19268, 0.262342, 0.688198, -3.00953], [0.227837, 0.105274, 0.714764, -3.24167], [0.263756, -0.0584146, 0.717896, -3.27098], [0.2997, -0.219308, 0.716109, -3.13132], [0.335743, -0.369781, 0.722898, -2.85883], [0.372459, -0.503863, 0.74431, -2.48155] … [-0.154043, 2.62266, 0.0918259, 1.92926], [-0.145452, 2.7147, 0.2526, 1.76946], [-0.129953, 2.79896, 0.368201, 1.61626], [-0.109697, 2.87565, 0.442902, 1.46555], [-0.0866524, 2.94479, 0.479719, 1.31292], [-0.062689, 3.00611, 0.479628, 1.15119], [-0.0396983, 3.05883, 0.44074, 0.967804], [-0.019767, 3.10136, 0.357141, 0.741565], [-0.00542273, 3.1307, 0.217085, 0.437716], [0.0, 3.14159, 0.0, -4.96525e-28]]","[[5.32145], [3.21081], [1.62781], [0.543559], [0.00326584], [-0.00820181], [0.375279], [0.944423], [1.49879], [1.89843] … [5.12628], [3.98779], [2.95112], [2.0094], [1.13648], [0.291712], [-0.581017], [-1.55933], [-2.75481], [-4.33465]]",421.899,0.845,LOCALLY_SOLVED
4,"[0.0089636, 1.46611]","[[0.0089636, 1.46611, 0.0, 0.0], [0.0182213, 1.43971, 0.371497, -1.05882], [0.0455334, 1.35987, 0.724641, -2.1433], [0.0895713, 1.22507, 1.04296, -3.2598], [0.147653, 1.03487, 1.28756, -4.35521], [0.214825, 0.793843, 1.40357, -5.27411], [0.283527, 0.51656, 1.34027, -5.77095], [0.344819, 0.228481, 1.09918, -5.6775], [0.391747, -0.0428146, 0.766153, -5.10423], [0.422421, -0.279853, 0.454608, -4.33089] … [-0.211826, 2.34593, -0.214524, 2.59743], [-0.214631, 2.47058, 0.102795, 2.41333], [-0.20299, 2.58726, 0.36366, 2.2768], [-0.17979